## Setup

In [9]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import pandas as pd

from harp_resources import process, utils
from sleap import load_and_process as lp

In [50]:
#data_path = Path('/home/ikharitonov/RANCZLAB-NAS/data/ONIX/20240730_Mismatch_Experiment/MMclosed&Regular_120824/2024-08-12T16-51-16_B3M3')
#photometry_path = Path('/home/ikharitonov/RANCZLAB-NAS/data/ONIX/20240730_Mismatch_Experiment/MMclosed&Regular_120824/photometry/B3M3_MMclosed&Regular_day1/2024_08_12-18_57_17')


In [51]:
rootdir = '/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_080824' #Enter root path
data_paths = []
photometry_paths= []
for dirpath, subdirs, files in os.walk(rootdir):
    for x in files:
        if 'ExperimentEvents' in x:
            data_paths.append(dirpath[:-17])
        if 'Fluorescence.csv' in x:
            photometry_paths.append(dirpath)




In [52]:
'2024-08-08T08-22-12_B2M5'[20:]

'B2M5'

In [53]:
def run_extraction_makeh5(data_path, photometry_path, make_h5 = False):
    print(f'\n Running extraction for {data_path.split('/')[-1][20:]} for session {data_path.split('/')[-2]}')
    data_path = Path(data_path)
    photometry_path = Path(photometry_path)
    SessionSettings = utils.read_SessionSettings(data_path, print_contents=True)
    conversions = process.calculate_conversions_second_approach(data_path, photometry_path, verbose=False)
    streams = utils.load_registers(data_path)

    Photometry = utils.read_fluorescence(photometry_path)
    Photometry['HARP Timestamps'] = conversions['photometry_to_harp_time'](Photometry['TimeStamp'])
    
    OnixAnalogClock = utils.read_OnixAnalogClock(data_path)
    OnixAnalogData = utils.read_OnixAnalogData(data_path, binarise=True)
    ExperimentEvents = utils.read_ExperimentEvents(data_path) 
    
    photodiode_series = pd.Series(OnixAnalogData[:,0], index=conversions['onix_to_harp_timestamp'](OnixAnalogClock))

    print('Adding Photometry, Eye Movements and Photodiode to the streams')
    streams = process.reformat_and_add_many_streams(streams, Photometry, 'Photometry', ['CH1-410', 'CH1-470', 'CH1-560'], index_column_name='HARP Timestamps')
    streams = process.add_stream(streams, 'ONIX', photodiode_series, 'Photodiode')
    
    
    _ = process.get_timepoint_info(streams, print_all=True)
    resampled_streams = process.pad_and_resample(streams, resampling_period='1 ms', method='linear')
    _ = process.get_timepoint_info(resampled_streams, print_all=True)

    print('Applying linear and angular conversion to Optical tracking sensor streams (cm / sec and degrees / sec)')
    resampled_streams['H1']['OpticalTrackingRead0X(46)'] = process.running_unit_conversion(resampled_streams['H1']['OpticalTrackingRead0X(46)']*100)
    resampled_streams['H1']['OpticalTrackingRead0Y(46)'] = process.rotation_unit_conversion(resampled_streams['H1']['OpticalTrackingRead0Y(46)'])

    print('Streams are extracted and can be used or made to h5')
    if make_h5:
        process.save_streams_as_h5(data_path, resampled_streams, streams_to_save_pattern)


In [54]:

for path in photometry_paths:
    if data_path[len(data_path)-4:] in path:
        photometry_path = Path(path)

data_path = Path(data_path)

print(data_path, '\n', photometry_path)

TypeError: object of type 'PosixPath' has no len()

In [ ]:
for data, photometry in zip(data_paths, photometry_paths):
    if data[len(data)-4:] in photometry:
        print(data, photometry)
        run_extraction_makeh5(data, photometry)

In [38]:
type(photometry_paths[1])

str

## Loading and Synchronisation 

In [55]:
conversions = process.calculate_conversions_second_approach(data_path, photometry_path, verbose=False)
# After hardware ONIX clock implementation - this will have to be adapted
# Only photometry will need to be converted

OnixAnalogClock loaded in 6.44 seconds.


/Users/hildeteigen/Documents/GitHub/vestibular_vr_pipeline/harp_resources/utils.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sorted_filenames = pd.to_datetime(pd.Series([x.split('_')[1].split('.')[0] for x in filenames])).sort_values()



Shape of OnixAnalogClock == [218759300] shape of OnixAnalogFrameCount == [218764700].
Cutting 5400 values from the beginning of OnixAnalogFrameCount. Data considered to be MISSING.

OnixDigital has 707 events and Events.csv has 698 events. Cutting to the minimum 698 events from the beggining, aligning by the end.

Calculation of conversions finished in 81.39 seconds.


/Users/hildeteigen/Documents/GitHub/vestibular_vr_pipeline/harp_resources/utils.py:66: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sorted_filenames = pd.to_datetime(pd.Series([x.split('_')[1].split('.')[0] for x in filenames])).sort_values()


In [56]:
streams = utils.load_registers(data_path)

Photometry = utils.read_fluorescence(photometry_path)
Photometry['HARP Timestamps'] = conversions['photometry_to_harp_time'](Photometry['TimeStamp'])

OnixAnalogClock = utils.read_OnixAnalogClock(data_path)
OnixAnalogData = utils.read_OnixAnalogData(data_path, binarise=True)
ExperimentEvents = utils.read_ExperimentEvents(data_path) 

photodiode_series = pd.Series(OnixAnalogData[:,0], index=conversions['onix_to_harp_timestamp'](OnixAnalogClock))

Dataset 2024-08-08T08-22-12_B2M5 contains following registers:
H1: [32, 33, 46]
H2: [39, 42]
Registers loaded in 3.33 seconds.
OnixAnalogClock loaded in 5.88 seconds.
OnixAnalogData loaded in 162.24 seconds.


/Users/hildeteigen/Documents/GitHub/vestibular_vr_pipeline/harp_resources/utils.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sorted_filenames = pd.to_datetime(pd.Series([x.split('_')[1].split('.')[0] for x in filenames])).sort_values()


In [57]:
# Adding Photometry, Eye Movements and Photodiode to the streams
streams = process.reformat_and_add_many_streams(streams, Photometry, 'Photometry', ['CH1-410', 'CH1-470', 'CH1-560'], index_column_name='HARP Timestamps')
streams = process.add_stream(streams, 'ONIX', photodiode_series, 'Photodiode')

In [64]:
_ = process.get_timepoint_info(streams, print_all=True)
resampled_streams = process.pad_and_resample(streams, resampling_period='1 ms', method='linear')
_ = process.get_timepoint_info(resampled_streams, print_all=True)

Global first timestamp: 1904-01-09 19:54:29.679488
Global last timestamp: 1904-01-09 20:31:03.269787765
Global length: 0 days 00:36:33.590299765

H1
Cam0Event(32): 
	first  1904-01-09 19:54:29.680640 
	last   1904-01-09 20:31:00.971808 
	length 0 days 00:36:31.291168 
	mean difference between timestamps 0 days 00:00:00.016666092
Cam1Event(33): 
	first  1904-01-09 19:54:29.680640 
	last   1904-01-09 20:31:00.971808 
	length 0 days 00:36:31.291168 
	mean difference between timestamps 0 days 00:00:00.016666092
OpticalTrackingRead0X(46): 
	first  1904-01-09 19:54:29.680256 
	last   1904-01-09 20:31:00.970240 
	length 0 days 00:36:31.289984 
	mean difference between timestamps 0 days 00:00:00.009999999
OpticalTrackingRead0Y(46): 
	first  1904-01-09 19:54:29.680256 
	last   1904-01-09 20:31:00.970240 
	length 0 days 00:36:31.289984 
	mean difference between timestamps 0 days 00:00:00.009999999
OpticalTrackingRead0Brightness(46): 
	first  1904-01-09 19:54:29.680256 
	last   1904-01-09 20:31:0

In [65]:
# Applying linear and angular conversion to Optical tracking sensor streams
# OpticalTrackingRead0X(46) converted to centimeters per second
# OpticalTrackingRead0Y(46) covnerted to degrees per second
resampled_streams['H1']['OpticalTrackingRead0X(46)'] = process.running_unit_conversion(resampled_streams['H1']['OpticalTrackingRead0X(46)']*100)
resampled_streams['H1']['OpticalTrackingRead0Y(46)'] = process.rotation_unit_conversion(resampled_streams['H1']['OpticalTrackingRead0Y(46)'])

In [67]:
print(type(resampled_streams['Photometry']['CH1-470']))
resampled_streams['Photometry']['CH1-470']

<class 'pandas.core.series.Series'>


1904-01-09 19:54:29.679     0.000000
1904-01-09 19:54:29.680     0.001054
1904-01-09 19:54:29.681     0.002108
1904-01-09 19:54:29.682     0.003161
1904-01-09 19:54:29.683     0.004215
                             ...    
1904-01-09 20:31:03.265    44.381788
1904-01-09 20:31:03.266    44.376091
1904-01-09 20:31:03.267    44.370394
1904-01-09 20:31:03.268    44.364697
1904-01-09 20:31:03.269    44.359000
Freq: ms, Length: 2193591, dtype: float64

In [19]:
streams_to_save_pattern = {'H1': ['OpticalTrackingRead0X(46)', 'OpticalTrackingRead0Y(46)'], 'H2': ['Encoder(38)'], 'Photometry': ['CH1-410', 'CH1-470', 'CH1-560'], 'ONIX': ['Photodiode']}
streams_to_save_pattern


{'H1': ['OpticalTrackingRead0X(46)', 'OpticalTrackingRead0Y(46)'],
 'H2': ['Encoder(38)'],
 'Photometry': ['CH1-410', 'CH1-470', 'CH1-560'],
 'ONIX': ['Photodiode']}